# Advent of Code Day 3

Day 3 was a fairly challenging one even though the problem is straight-forward enough to explain.  A square is numbered 1.  Then each square radiating around it in a counter-clockwise circle is monotonically ordered accordingly.  In part one, the objective is to determine the distance to move from square 347991 to square 1 using Manhattan distance.  In part two, the squares, instead of being simply numbered, are assigned a value based on the value of the adjacent squares (again starting at 1) and the objective is to find the first assigned value greater than the value of 347991.  

In [ ]:
from utils import read_input

### Coordinate Square

The core of this problem is to produce a grid of numbered squares according to the rules.  I chose to orient square 1 at coordinate (0,0) (conceptually the Cartestian plane origin).  By doing so, it's possible to do two things.  One: we can talk about squares in terms of "levels":  square 1 is level 1, squares 2 to 9 are level 2, etc.  Two: we are always maneuvering symmetrically about the origin square and so the absolute values of the squares is not that important in terms of moving around.  

After we've assigned square 1 to (0,0), we use a simple loop to work through as many levels as are desired.  The first move for a new level is to step to the right, then move up to the upper right corner, then across, then down, and then back to the bottom right corner.  Once the bottom right corner is reached, the level has ended.  We can use the level to control the moves along either axis.  Ultimately, we return a dictionary that maps a square number to a coordinate. 

In [ ]:
def create_coordinate_square(max_level = 6):
    
    coordinates = {}
    
    coordinates[1] = (0, 0)
    square = 2
    
    for level in xrange(1, max_level + 1):
        
        for y in xrange(-(level-1), level + 1):            
            coordinates[square] = (level, y)
            square+=1
            
        for x in xrange(level - 1, -(level + 1), -1):           
            coordinates[square] = (x, level)            
            square+=1
            
        for y in xrange(level - 1, -(level + 1), -1):
            coordinates[square] = (-level, y)            
            square+=1
            
        for x in xrange(-level + 1, level + 1):            
            coordinates[square] = (x, -level)            
            square+=1
    
    return coordinates
        
    
    
    

### Distance To

For part 1, we need to compute the distance from some square to the origin.  For Manhattan distance, that computation is simply the sum of the absolute value of the differences in distance in each dimension.  To make the function more general, this version requires the caller to pass in both the start and end square, along with the coordinate square in question (the square number --> coordinate map).  

In [ ]:
def distance_to(start_square, end_square, coordinate_square):
    
    x1, y1 = coordinate_square[start_square]
    
    x2, y2 = coordinate_square[end_square]
    
    return abs(x1-x2) + abs(y1-y2)
    
    
 

### Invert Dictionary

Inverting a dictionary (interchanging the keys and values) can be done with a simple dictionary comprehension, but I preferred having it in its own function. 

In [ ]:
def invert(d):   
    return {v:k for k,v in d.iteritems()}
    
    

### Squares Adjacent To

Given a square and both a dictionary of square numbers to coordinates AND the inversion (coordinates to square numbers), returns 
a list of the square numbers adjacent to square.  In the event we are at the boundary of the square (meaning we reach the outer level), then we simply don't include as adjacents any square that doesn't actually exist.  There were other options for handling this (including using NaN as the value when not all adjacents exist), but doing it like this made sense since the square can never be infinite anyway. 

In [ ]:
def squares_adjacent_to(square, square_to_coordinates, coordinates_to_square):
    current_x, current_y = square_to_coordinates[square]
    
    adjacent_squares = []
    
    # This part works but it throws a KeyError since we figure out the coordinates around square
    # without regard to whether those coordinates actually exist.  Have to either figure out
    # a way to stay away from those coordinates (or check them against something) or else
    # rework the functions based on the actual problem requirements.  
    for x in xrange(current_x-1, current_x+2):
        for y in xrange(current_y-1, current_y+2):
            
            if (x,y) in coordinates_to_square:
                adjacent = coordinates_to_square[(x, y)]
         
                if adjacent != square:
                    adjacent_squares.append(adjacent)
    
    return adjacent_squares
    
    

### Assign Square Values

Given a dict that maps square numbers to coordinates, this function assigns values to each numbered square (returning that relationship as a dict).  This is necessary for part 2.  

In [ ]:
def assign_square_values(square_to_coordinates):
    
    from collections import defaultdict    
    coordinates_to_square = invert(square_to_coordinates)
    
    square_values = defaultdict(lambda: 0)  # Assume any value not in the dict is 0 if we try to retrieve it.    
    square_values[1] = 1
    
    for square in xrange(2, max(square_to_coordinates.keys())):
        
        # What squares are adjacent to square?
        adjacent_squares = squares_adjacent_to(square, square_to_coordinates, coordinates_to_square)
        
        # What values are in the adjacent squares? 
        adjacent_values = [square_values[s] for s in adjacent_squares]        
        
        # Set the value for square to the sum of its adjacent values.  By using the defaultdict, we can automatically
        # treat as 0 any adjacent square we haven't assigned yet without special casing it.
        square_values[square] = sum(adjacent_values)
        
        
    return square_values
        
    
    
    

In [ ]:
def solve_part_one():
    coordinate_square = create_coordinate_square(max_level = 301)
    
    print 'Distance from square 347991 to origin = {}'.format(distance_to(347991, 1, coordinate_square))

In [ ]:
def solve_part_two():
    coordinate_square = create_coordinate_square(max_level = 50)
    
    value_squares = assign_square_values(coordinate_square)
    
    above_target = [v for k,v in value_squares.iteritems() if v > 347991]
    
    print 'First Value Assigned Greater Than 347991 = {}'.format(min(above_target))

In [ ]:
solve_part_one()

In [ ]:
solve_part_two()